Packages to be used

In [1]:
from ipynb.fs.full.funcs import *

In [2]:
crs = 'EPSG:3395'

Define the workspace. The output layers will populate this folder. It is highly recommended to select an empty folder as your workspace
        

In [3]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

Select the clusters to be used in the analysis'

Please also idicate which column is representing the population data. 

In [4]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    
popunit = widgets.Dropdown(options=clusters.head(),
    value=None,
    description='Population:',
    disabled=False)
display(popunit)

x= popunit.value

Dropdown(description='Population:', options=('id', 'NightLight', 'Country', 'Pop', 'GridCellAr', 'ElecPop', 'g…

Select the landcover map

In [22]:
clusters = processing_raster("landcover","majority",clusters)

2020-02-01 15:14:16.510103


Select the elevation map

In [ ]:
clusters = processing_elevation_and_slope("elevation","mean",clusters, workspace,crs)

Select the ghi map

In [ ]:
clusters = processing_raster("ghi","mean",clusters)

Select the traveltime map

In [ ]:
clusters = processing_raster("traveltime","mean",clusters)

Select the wind velocity map

In [ ]:
clusters = processing_raster("wind","mean",clusters)

Select the custom demand map (Optional)

In [ ]:
clusters = processing_raster("customdemand","mean",clusters)

Saving the clusters with extracted rasters

In [12]:
clusters = finalizing_rasters(workspace, clusters, crs)

2020-02-01 15:08:23.740409


preparing for vectors

In [5]:
clusters = preparing_for_vectors(workspace, clusters, crs)

2020-02-02 18:21:35.511608


Select the substations to be used in the analysis (Optional)

In [ ]:
clusters = processing_points("Substation", admin, crs, workspace, clusters)


Select the existing high voltage transmission lines (Optional)

In [ ]:
clusters = processing_lines("Existing_HV", admin, crs, workspace, clusters)

Select the planned high voltage transmission lines (Optional)


In [ ]:
clusters = processing_lines("Planned_HV", admin, crs, workspace, clusters)

Select the existing medium voltage distribution lines (Optional)

In [ ]:
clusters = processing_lines("Existing_MV", admin, crs, workspace, clusters)

Select the planned medium voltage distribution lines (Optional)

In [ ]:
clusters = processing_lines("Planned_MV", admin, crs, workspace, clusters)

Select the road network to be used (Optional)

In [ ]:
clusters = processing_lines("Roads", admin, crs, workspace, clusters)

Select the transformers to be used (Optional)

In [ ]:
#clusters = processing_points("Transformer", admin, crs, workspace, clusters)

Select the hydropower dataset to be used (Optional)

If a hydropower dataset is selected please identify the column represeting the hydro potential as well as its unit

In [6]:
messagebox.showinfo('OnSSET', 'Select the Hydropower map')
hydro=gpd.read_file(filedialog.askopenfilename(title = "Select Hydro map", filetypes = (("shapefile","*.shp"),("all files","*.*"))))

hydropower = widgets.Dropdown(options=hydro.head(),
    value=None,
    description='Hydropower:',
    disabled=False)

display(hydropower)

hydrounit = widgets.Dropdown(options=['W', 'kW', 'MW'],
    value='W',
    description='Unit:',
    disabled=False)

display(hydrounit)

Dropdown(description='Hydropower:', options=('OBJECTID_1', 'OBJECTID', 'river_ID', 'elev', 'head', 'discharge_…

Dropdown(description='Unit:', options=('W', 'kW', 'MW'), value='W')

In [37]:

points_clip = clip.clip_shp(hydro, admin)
points_clip.crs = {'init' :'epsg:4326'}
points_proj=points_clip.to_crs({ 'init': crs})

points_proj.to_file(workspace + r"\_proj.shp", driver='ESRI Shapefile')

points_f = fiona.open(workspace + r"\_proj.shp")
points = gpd.read_file(workspace +  r"\_proj.shp")
points2 = fiona.open(workspace + r'\clusters_cp.shp')

columns = list(points_f.meta["schema"]["properties"]) + ["geometry"]

#geoms1 = [shape(feat["geometry"]) for feat in points_f]

geoms1 = []
for each in points_f:
    geoms1.append(each["properties"])
    geoms1.append(each["geometry"])
    

s1_arr = np.array(geoms1)
#s1_arr = np.array(s1)"

#properties = [shape(feat["properties"]) for feat in points_f]

#hydro
#p = clip.clip_shp(hydro, admin)

#parr = p.to_numpy()

#parr

#geoms1 = [shape(feat[hydropower.value]) for feat in hydro]
#s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
#s1_arr = np.array(s1)

s1_arr

s1 = list(geoms1)
s2 = list(s1)
s2

s3 = np.array(list(s2))
s3

s1

geoms1 = []


for each in points_f:
    geoms1.append(each["properties"])
    

geoms1

[OrderedDict([('OBJECTID_1', 9970),
              ('OBJECTID', 9970),
              ('river_ID', 45957),
              ('elev', 2),
              ('head', 6.0),
              ('discharge_', 17.5918),
              ('power', 524851.0),
              ('PowerMW', 0.524851),
              ('ENUM_ID', 9970)]),
 OrderedDict([('OBJECTID_1', 15114),
              ('OBJECTID', 4898),
              ('river_ID', 46750),
              ('elev', 1),
              ('head', 4.0),
              ('discharge_', 470.636),
              ('power', 9360940.0),
              ('PowerMW', 9.36094),
              ('ENUM_ID', 15114)]),
 OrderedDict([('OBJECTID_1', 15159),
              ('OBJECTID', 4943),
              ('river_ID', 46018),
              ('elev', 5),
              ('head', 1.0),
              ('discharge_', 998.496),
              ('power', 4965020.0),
              ('PowerMW', 4.96501),
              ('ENUM_ID', 15159)]),
 OrderedDict([('OBJECTID_1', 15162),
              ('OBJECTID', 4946),
    

In [36]:
clusters = processing_hydro("Hydropower", admin, crs, workspace, clusters, hydro)

TypeError: float() argument must be a string or a number, not 'collections.OrderedDict'

In [8]:
clusters

,id,NightLight,Country,Pop,GridCellAr,ElecPop,geometry
0,1,0.00000,Gambia,72.73186,0.051,0.00000,POLYGON ((-1677521.408706525 1543831.897960086...
1,2,0.00000,Gambia,227.28708,0.077,0.00000,POLYGON ((-1676593.660531499 1543452.039613444...
2,3,0.00000,Gambia,245.47004,0.060,0.00000,POLYGON ((-1677892.507976536 1542692.339440355...
3,4,0.00000,Gambia,290.92746,0.076,0.00000,POLYGON ((-1673532.091553911 1542407.457552382...
4,5,0.00000,Gambia,36.36593,0.026,0.00000,POLYGON ((-1671676.595203858 1541932.661284465...
5,6,0.00000,Gambia,9.09148,0.009,0.00000,POLYGON ((-1672140.469291371 1541837.703062483...
6,7,0.00000,Gambia,327.29339,0.068,0.00000,POLYGON ((-1672326.018926377 1542027.619850285...
7,8,0.00000,Gambia,172.73818,0.043,0.00000,"POLYGON ((-1670006.64848881 1541457.873612458,..."
8,9,0.00000,Gambia,181.82966,0.093,0.00000,"POLYGON ((-1676037.011626482 1541267.96094987,..."
9,10,0.00000,Gambia,272.74449,0.085,0.00000,"POLYGON ((-1683458.997026695 1541267.96094987,..."


In [ ]:
#conditioning()